<h3>Stemming, rimozione stopwords e normalizzazione</h3>
<p>In questo notebook proveremo ad addestrare i modelli utilizzando gli 'n-grams'. Al contrario della rappresentazione 'bag-of-words', in cui i singoli lessemi del linguaggio vengono utilizzati come features, utilizzeremo come fetures sotto sequenze di parole contigue di lunghezza n.<br>
L'idea alla base di questo approcio è che vengano preservate maggiormente le informazioni relative al contesto in cui le singole parole vengono utilizzate.

In [ ]:
import os
import time
import re
import numpy as np
import pandas as pd

#Libreria di ML per Python, oltre ad offrire innumerevoli implementazioni di modelli predittivi offre anche
#una vasta gamma di funzionalità per il data cleaning, feature selection, feature extraction, etc...
#Per ora impiegheremo la classe CountVectorizer per creare la 'bag of words'.
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.svm import SVR

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

#Per poter utilizzare la lista di stopwords di NLTK è necessario scaricarla!
#import nltk
#nltk.download()

#Libreria per generare grafici
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import ColumnDataSource, LabelSet, FactorRange

#Funzione invocata per redirigere l'output della libreria
#Bokeh in modo da visualizzare i grafici nel notebook, se omesso i grafici sono generati sotto forma
#di file HTML
output_notebook()

SEED = 99
np.random.seed(SEED)

DATA_FOLDER = os.path.join(os.getcwd(), "../data")
RESULTS_FOLDER = os.path.join(DATA_FOLDER, "results")
CV_RESULTS_FOLDER = os.path.join(RESULTS_FOLDER, "csv")

<h3>Funzioni ausiliarie</h3>

In [ ]:
def clean_results_df(df, to_remove, to_format, format_, new_index_cols):
    """
        Clean results dataframe contaning GridSearch error metrics.

        Params :
            df -> pandas.DataFrame : DataFrame to clean.
            to_remove -> python.dict : dictionary of elements to remove.
            new_index_cols -> python.list : list of columns to use as new index
    """
    df = df.copy()
    for k,v in to_remove.items():
        if v is True:
            df = df.drop(k, axis=1)
        elif isinstance(v, list):
            df = df.drop(k, 1).assign(**pd.DataFrame(df[k].values.tolist()))
            for col in v:
                df = df.drop(col, 1)
    labels = [df[col] for col in new_index_cols]
    labels = list(zip(*labels))
    new_index = pd.MultiIndex.from_tuples(labels, names=new_index_cols)
    df = df.set_index(new_index)
    df = df.drop(new_index_cols, 1)
    for col in to_format:
        df[col] = df[col].apply(lambda x : format(x, format_))
    return df

def dump_results(df, fp):
    df.to_csv(fp)

<h3>Lettura Dati</h3>

In [ ]:
train_fp = os.path.join(os.getcwd(), "../data/csv/train.csv")
train = pd.read_csv(
    train_fp,
    index_col=['review_id']
)

In [ ]:
test_fp = os.path.join(os.getcwd(), "../data/csv/train.csv")
test = pd.read_csv(
    test_fp,
    index_col=['review_id']
)

<h3>Separazione feature-labels</h3>

In [ ]:
X_train, Y_train = train['text'], train['stars']
X_test, Y_test = test['text'], test['stars']

<h3>Definizione della Pipeline</h3>

In [ ]:
pipeline = Pipeline([
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('model', 'passthrough')
])

<h3>Griglia iperparametri</h3>

In [ ]:
param_grid = [
    {
        'cv__ngram_range':[(1,2), (2,2), (3,3)],
        'tfidf__norm' : ['l2', 'l1'],
        'tfidf__smooth_idf' : [True, False],
        'model':[SGDRegressor()],
        'model__penalty':['l2'],#,'l1'],
        'model__alpha':[0.00001, 0.0001, 0.001],
        'model__max_iter':[3500],
        'model__random_state':[SEED]
    },
    {
        'cv__ngram_range':[(1,2), (2,2), (3,3)],
        'tfidf__smooth_idf' : [True, False],
        'model':[DecisionTreeRegressor()],
        'model__splitter':['random'],
        'model__max_depth':[70, 100, 130],
        'model__min_samples_split':[800, 1000],
        'model__min_samples_leaf':[400, 500, 600],
        'model__max_features':[5000, 7000],
        'model__random_state': [SEED]
    },
    {
        'cv__ngram_range':[(1,3)],
        'tfidf__smooth_idf' : [True, False],
        'model':[SVR()],
        'model__kernel':['linear', 'poly', 'rbf'],
        'model__degree':[2, 5, 10],
        'model__gamma':['scale', 'auto'],
        'model__C':[1, 10, 50],
        'model__max_iter':[3500]
    }
]

In [ ]:
piepeline_gs_cv = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring=['neg_mean_absolute_error', 'neg_root_mean_squared_error'],
    n_jobs=-1,
    cv=3,
    refit=False
)

In [ ]:
piepeline_gs_cv.fit(X_train, Y_train)

<h3>Dump risultati GridSearchCV</h3>

In [ ]:
results_df = clean_results_df(
    pd.DataFrame(piepeline_gs_cv.cv_results_),
    to_remove={},
    to_format=['mean_test_neg_mean_absolute_error', 'mean_test_neg_root_mean_squared_error'],
    format_=".8f",
    new_index_cols=['rank_test_neg_root_mean_squared_error', 'rank_test_neg_mean_absolute_error']
)

In [ ]:
results_fp = os.path.join(CV_RESULTS_FOLDER, "ngrams_cv_results.csv")
dump_results(results_df, results_fp)